# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f279d6b29a0>
├── 'a' --> tensor([[ 0.3836,  1.0910, -1.2552],
│                   [-0.3795,  0.7519, -1.1782]])
└── 'x' --> <FastTreeValue 0x7f279d6b2be0>
    └── 'c' --> tensor([[-1.0088, -0.3954,  1.7330, -0.2671],
                        [ 0.1175, -0.2248, -0.5570,  0.8946],
                        [-0.3011, -0.1424,  1.9044,  1.2785]])

In [4]:
t.a

tensor([[ 0.3836,  1.0910, -1.2552],
        [-0.3795,  0.7519, -1.1782]])

In [5]:
%timeit t.a

59.7 ns ± 0.0884 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f279d6b29a0>
├── 'a' --> tensor([[-0.2857,  0.2692,  0.5676],
│                   [ 0.3606, -0.2112, -0.8537]])
└── 'x' --> <FastTreeValue 0x7f279d6b2be0>
    └── 'c' --> tensor([[-1.0088, -0.3954,  1.7330, -0.2671],
                        [ 0.1175, -0.2248, -0.5570,  0.8946],
                        [-0.3011, -0.1424,  1.9044,  1.2785]])

In [7]:
%timeit t.a = new_value

60.4 ns ± 0.0632 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3836,  1.0910, -1.2552],
               [-0.3795,  0.7519, -1.1782]]),
    x: Batch(
           c: tensor([[-1.0088, -0.3954,  1.7330, -0.2671],
                      [ 0.1175, -0.2248, -0.5570,  0.8946],
                      [-0.3011, -0.1424,  1.9044,  1.2785]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3836,  1.0910, -1.2552],
        [-0.3795,  0.7519, -1.1782]])

In [11]:
%timeit b.a

52.3 ns ± 0.0524 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2560,  0.2669,  0.2726],
               [-0.5459,  1.2236,  0.2685]]),
    x: Batch(
           c: tensor([[-1.0088, -0.3954,  1.7330, -0.2671],
                      [ 0.1175, -0.2248, -0.5570,  0.8946],
                      [-0.3011, -0.1424,  1.9044,  1.2785]]),
       ),
)

In [13]:
%timeit b.a = new_value

486 ns ± 0.123 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

846 ns ± 15.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 36 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 313 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 397 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f26ea55b4f0>
├── 'a' --> tensor([[[ 0.3836,  1.0910, -1.2552],
│                    [-0.3795,  0.7519, -1.1782]],
│           
│                   [[ 0.3836,  1.0910, -1.2552],
│                    [-0.3795,  0.7519, -1.1782]],
│           
│                   [[ 0.3836,  1.0910, -1.2552],
│                    [-0.3795,  0.7519, -1.1782]],
│           
│                   [[ 0.3836,  1.0910, -1.2552],
│                    [-0.3795,  0.7519, -1.1782]],
│           
│                   [[ 0.3836,  1.0910, -1.2552],
│                    [-0.3795,  0.7519, -1.1782]],
│           
│                   [[ 0.3836,  1.0910, -1.2552],
│                    [-0.3795,  0.7519, -1.1782]],
│           
│                   [[ 0.3836,  1.0910, -1.2552],
│                    [-0.3795,  0.7519, -1.1782]],
│           
│                   [[ 0.3836,  1.0910, -1.2552],
│                    [-0.3795,  0.7519, -1.1782]]])
└── 'x' --> <FastTreeValue 0x7f26ea55b340>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 61 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f26e38b5550>
├── 'a' --> tensor([[ 0.3836,  1.0910, -1.2552],
│                   [-0.3795,  0.7519, -1.1782],
│                   [ 0.3836,  1.0910, -1.2552],
│                   [-0.3795,  0.7519, -1.1782],
│                   [ 0.3836,  1.0910, -1.2552],
│                   [-0.3795,  0.7519, -1.1782],
│                   [ 0.3836,  1.0910, -1.2552],
│                   [-0.3795,  0.7519, -1.1782],
│                   [ 0.3836,  1.0910, -1.2552],
│                   [-0.3795,  0.7519, -1.1782],
│                   [ 0.3836,  1.0910, -1.2552],
│                   [-0.3795,  0.7519, -1.1782],
│                   [ 0.3836,  1.0910, -1.2552],
│                   [-0.3795,  0.7519, -1.1782],
│                   [ 0.3836,  1.0910, -1.2552],
│                   [-0.3795,  0.7519, -1.1782]])
└── 'x' --> <FastTreeValue 0x7f26e38b50a0>
    └── 'c' --> tensor([[-1.0088, -0.3954,  1.7330, -0.2671],
                        [ 0.1175, -0.2248, -0.5570,  0.8946],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.3 µs ± 206 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.0088, -0.3954,  1.7330, -0.2671],
                       [ 0.1175, -0.2248, -0.5570,  0.8946],
                       [-0.3011, -0.1424,  1.9044,  1.2785]],
              
                      [[-1.0088, -0.3954,  1.7330, -0.2671],
                       [ 0.1175, -0.2248, -0.5570,  0.8946],
                       [-0.3011, -0.1424,  1.9044,  1.2785]],
              
                      [[-1.0088, -0.3954,  1.7330, -0.2671],
                       [ 0.1175, -0.2248, -0.5570,  0.8946],
                       [-0.3011, -0.1424,  1.9044,  1.2785]],
              
                      [[-1.0088, -0.3954,  1.7330, -0.2671],
                       [ 0.1175, -0.2248, -0.5570,  0.8946],
                       [-0.3011, -0.1424,  1.9044,  1.2785]],
              
                      [[-1.0088, -0.3954,  1.7330, -0.2671],
                       [ 0.1175, -0.2248, -0.5570,  0.8946],
                       [-0.3011, -0.1424,  1.9044,  1.2785]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.0088, -0.3954,  1.7330, -0.2671],
                      [ 0.1175, -0.2248, -0.5570,  0.8946],
                      [-0.3011, -0.1424,  1.9044,  1.2785],
                      [-1.0088, -0.3954,  1.7330, -0.2671],
                      [ 0.1175, -0.2248, -0.5570,  0.8946],
                      [-0.3011, -0.1424,  1.9044,  1.2785],
                      [-1.0088, -0.3954,  1.7330, -0.2671],
                      [ 0.1175, -0.2248, -0.5570,  0.8946],
                      [-0.3011, -0.1424,  1.9044,  1.2785],
                      [-1.0088, -0.3954,  1.7330, -0.2671],
                      [ 0.1175, -0.2248, -0.5570,  0.8946],
                      [-0.3011, -0.1424,  1.9044,  1.2785],
                      [-1.0088, -0.3954,  1.7330, -0.2671],
                      [ 0.1175, -0.2248, -0.5570,  0.8946],
                      [-0.3011, -0.1424,  1.9044,  1.2785],
                      [-1.0088, -0.3954,  1.7330, -0.2671],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 502 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

305 µs ± 1.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
